In [1]:
#Importer les bibliothèques nécessaires (pandas et pyspark) dans le script pyspark
import pandas as pd

In [2]:
import openpyxl

In [3]:
import pyspark.sql
from pyspark.sql import *

In [19]:
import requests

In [25]:
from pyspark import SparkFiles

In [31]:
import zipfile
import io
import os

In [4]:
#Création de la session spark 
spark = SparkSession.builder.appName("Excel to DataFrame").getOrCreate() 

In [52]:
def TelechargementData():

    url = "https://services.eaufrance.fr/documents/openData/SISPEA_FR_"+str(annee)+"_AEP.zip"

    reponse = requests.get(url)

    content = reponse.content

    # Création d'un objet ZipFile à partir du contenu ZIP téléchargé
    zip_file = zipfile.ZipFile(io.BytesIO(content))

    # Création du répertoire de destination s'il n'existe pas
    os.makedirs("Data", exist_ok=True)

    # Extraction des fichiers du fichier ZIP et enregistrement dans le répertoire de destination
    zip_file.extractall("Data")
    
    # Fermeture du fichier ZIP
    zip_file.close()

In [16]:
def lecturePandas(annee):
    ''' Cette fonction permet de lire un fichier excel et de le convertir en dataFrame Pandas'''

    #Pour lire le fichier excel on utilise Pandas 
    #On donne le chemin du fichier excel
    excel_file_path = "./Data/SISPEA_FR_"+str(annee)+"_AEP.xls"
    #On donne le nom de l'onglet
    sheet_name = "Entités de gestion"
    #Lecture du fichier excel avec pandas 
    pandas_df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

    return pandas_df

Je supprime la colonne fin de contrat, car elle creer un conflit qui ne me permet pas de convertir mon dataframe en table spark.

In [6]:
def supprimerLaColonneDateFin(pandas_df):
    '''Cette fonction supprime la colonne dateFinDeContrat dans mon dataframe Pandas'''
    pandas_df.drop(["Date de fin de contrat"], axis=1, inplace=True)

    return pandas_df

In [7]:
def convertirPandasEnSpark(pandas_df):
    '''Cette fonction converti la table pandas en table spark'''
    #Conversion du DataFrame Pandas en DataFrame Spark
    table = spark.createDataFrame(pandas_df)
    
    return table

In [8]:
def choixColonnes(table_spark):
    ''' Cette fonction selectionne les colonnes N° SIREN,VP.224,VP.225,VP.226,VP.227,VP.228,VP.229,VP.231,VP.232,VP.234 '''
    
    table_bonne_colonne=table_spark.select("N° SIREN","`VP.224`","`VP.225`", "`VP.226`", "`VP.227`","`VP.228`","`VP.229`","`VP.231`", "`VP.232`","`VP.234`")

    return table_bonne_colonne

Je renomme les colonnes de mon tableau

In [9]:
def renommageColonnes(table_spark):
    ''' Cette fonction renomme les colonnes de la table '''

    table_renamed = table_spark.withColumnRenamed("VP.224", "Indice linéaire de consommation")\
        .withColumnRenamed("VP.225", "Rendement sur les 3 années précédentes")\
        .withColumnRenamed("VP.226", "Rendement seuil par défaut")\
        .withColumnRenamed("VP.227", "Rendement seuil en ZRE")\
        .withColumnRenamed("VP.228", "Densité linéaire d'abonnés")\
        .withColumnRenamed("VP.229", "Ratio habitants par abonnés")\
        .withColumnRenamed("VP.231", "Consommation moyenne par abonné")\
        .withColumnRenamed("VP.232", "Volumes consommés comptabilisés")\
        .withColumnRenamed("VP.234", "Volume produit + Volume importé")
    return table_renamed

Je sauvegarde mon tableau dans un fichier csv

In [45]:
def transformationCSV(table_spark):
    ''' Cette fonction enregistre une table spark au format CSC '''
    table_spark.write.csv("consommation_eau.csv", header=True, mode="append")

In [54]:
def main(annee):
    
    TelechargementData()
    
    pandas_dataframe = lecturePandas(annee)

    pandas_dataframe_propre = supprimerLaColonneDateFin(pandas_dataframe)

    spark_dataframe = convertirPandasEnSpark(pandas_dataframe_propre)

    df_bonnes_colonnes = choixColonnes(spark_dataframe)

    df_bons_noms = renommageColonnes(df_bonnes_colonnes)

    transformationCSV(df_bons_noms)

In [51]:
# Selection des années à traiter.
annee_min = 2008
anne_max = 2021

In [55]:
#Execution du programme

for annee in range(annee_min,anne_max+1):
    main(annee)